In [1]:
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('../lib/')
import matplotlib.pyplot as plt
import math


import workers as workers
import sweden as sweden

In [2]:
result = pd.read_csv('../results/flows_1km_10km_sweden.csv')
grids = gpd.read_file("../results/grids_sweden/grids_1km_density_deso_with_10km_upper_grids.shp")

In [3]:
result.head()

,ozone,dzone,d_ij,D_ij_data,D_ij_sim,v_ij,grid_type
0,18,18,0.000000,0.000000,0.000000,0.000000,1
1,18,8,2.236068,1.639817,1.457747,43.436840,1
2,18,20,2.000000,1.648406,1.484936,54.296050,1
3,18,19,1.000000,1.960368,1.740077,217.184198,1
4,18,1,4.472136,1.534267,1.338410,10.859210,1


In [4]:
gdf_zones = gpd.read_file("../dbs/sweden/zones/DeSO/DeSO_2018_v2.shp")

In [5]:
grids.head()

,zone,deso,density,upper_deso,upper_dens,upper_zone,upper_xcoo,upper_ycoo,geometry
0,18,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((660000.000 6596000.000, 660000.000 6..."
1,8,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((659000.000 6598000.000, 659000.000 6..."
2,20,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((660000.000 6598000.000, 660000.000 6..."
3,19,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((660000.000 6597000.000, 660000.000 6..."
4,1,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((658000.000 6600000.000, 658000.000 6..."


In [6]:
agg_level=4

In [7]:
grids_dict = dict(zip(grids.zone, grids.deso))

In [8]:
result.loc[:,'ozone_deso'] = result.loc[:, 'ozone'].map(grids_dict).apply(lambda x: x[:agg_level])

In [9]:
result.loc[:,'dzone_deso'] = result.loc[:, 'dzone'].map(grids_dict).apply(lambda x: x[:agg_level])

In [10]:
result

,ozone,dzone,d_ij,D_ij_data,D_ij_sim,v_ij,grid_type,ozone_deso,dzone_deso
0,18,18,0.000000,0.000000,0.000000,0.000000,1,0114,0114
1,18,8,2.236068,1.639817,1.457747,43.436840,1,0114,0114
2,18,20,2.000000,1.648406,1.484936,54.296050,1,0114,0114
3,18,19,1.000000,1.960368,1.740077,217.184198,1,0114,0114
4,18,1,4.472136,1.534267,1.338410,10.859210,1,0114,0114
...,...,...,...,...,...,...,...,...,...
72778292,5028,5024,14.142136,1.404068,1.234242,88.768872,10,0136,0136
72778293,5028,5025,10.000000,1.434981,1.263294,177.537744,10,0136,0136
72778294,5028,5026,14.142136,1.404068,1.234242,88.768872,10,0136,0136
72778295,5028,5027,10.000000,1.434981,1.263294,177.537744,10,0136,0136


In [11]:
result_1 = result.groupby(['ozone_deso', 'dzone_deso'])[ 'v_ij'].sum().reset_index()

In [12]:
result_1

,ozone_deso,dzone_deso,v_ij
0,0114,0114,1.328431e+08
1,0114,0115,2.557794e+07
2,0114,0117,4.094059e+06
3,0114,0120,1.368843e+07
4,0114,0123,4.911734e+05
...,...,...,...
2117,2583,2518,5.234224e+03
2118,2583,2583,9.534196e+05
2119,2584,2521,4.155623e+03
2120,2584,2523,6.283611e+03


In [13]:
grids_big = gpd.read_file("../dbs/sweden/zones/Deso/DeSO_2018_v2.shp")

In [14]:
grids_big.head()

,id,objectid,uuid,deso,befolkning,version,geometry
0,1.0,1,{D9661502-EE24-45AE-A1F7-C1C80F463556},0114A0010,790,2018_v2,"POLYGON ((661116.252 6606615.603, 661171.409 6..."
1,2.0,2,{5EA47309-A001-4E2A-843E-02EC8F544E4C},0114C1010,1608,2018_v2,"POLYGON ((666960.066 6598800.393, 666971.371 6..."
2,3.0,3,{0DA292ED-A306-4D7E-A392-BD063A3093E1},0114C1020,1610,2018_v2,"POLYGON ((667034.814 6600076.634, 667032.984 6..."
3,4.0,4,{18583EFC-ECC1-4D27-B520-DA294F5658FD},0114C1030,2365,2018_v2,"POLYGON ((667095.512 6599103.521, 667095.009 6..."
4,5.0,5,{6574A7B3-AD12-4A81-8EAC-773B976B72BC},0114C1040,2346,2018_v2,"POLYGON ((664610.264 6600922.821, 664638.910 6..."


In [15]:
grids_big.loc[:,'upper_deso'] = grids_big.loc[:, 'deso'].apply(lambda x: x[0:4])

In [16]:
grids_big_use = grids_big.dissolve(by='upper_deso', aggfunc='sum')

In [17]:
grids_big_use

,geometry,id,objectid,befolkning
upper_deso,,,,
0114,"POLYGON ((664663.631 6597477.443, 664653.118 6...",276.0,276,46786
0115,"POLYGON ((675775.690 6600118.275, 675631.701 6...",627.0,627,34090
0117,"POLYGON ((686822.565 6592520.451, 686041.700 6...",1242.0,1242,45574
0120,"POLYGON ((697939.414 6569646.878, 697846.459 6...",1860.0,1860,45000
0123,"POLYGON ((659903.355 6587815.099, 659877.356 6...",4380.0,4380,79990
...,...,...,...,...
2580,"POLYGON ((800857.314 7287900.809, 799591.866 7...",271147.0,271147,78105
2581,"POLYGON ((793300.542 7241240.406, 793295.100 7...",166082.0,166082,42281
2582,"POLYGON ((811093.111 7304243.942, 811106.486 7...",107181.0,107181,28080


In [18]:
upper_deso_name = list(grids_big_use.index.array)

In [19]:
dis = dict()
for i in upper_deso_name:
    element = dict()
    for j in upper_deso_name:
        delta_x = grids_big_use.geometry.loc[i].centroid.x -  grids_big_use.geometry.loc[j].centroid.x
        delta_y = grids_big_use.geometry.loc[i].centroid.y -  grids_big_use.geometry.loc[j].centroid.y
        distance = delta_x * delta_x + delta_y * delta_y
        element[j] = pow(distance, 0.5)
    dis[i] = element

In [20]:
result_1.loc[:,'distance'] = 0

In [21]:
result_1.head()

,ozone_deso,dzone_deso,v_ij,distance
0,0114,0114,1.328431e+08,0
1,0114,0115,2.557794e+07,0
2,0114,0117,4.094059e+06,0
3,0114,0120,1.368843e+07,0
4,0114,0123,4.911734e+05,0


In [22]:
for i in result_1.index:
    result_1.loc[i, 'distance'] = dis[result_1.loc[i, 'ozone_deso']][result_1.loc[i, 'dzone_deso']]

In [23]:
result_1

,ozone_deso,dzone_deso,v_ij,distance
0,0114,0114,1.328431e+08,0.000000
1,0114,0115,2.557794e+07,18159.440980
2,0114,0117,4.094059e+06,36988.166911
3,0114,0120,1.368843e+07,56218.609477
4,0114,0123,4.911734e+05,10968.190328
...,...,...,...,...
2117,2583,2518,5.234224e+03,77605.749310
2118,2583,2583,9.534196e+05,0.000000
2119,2584,2521,4.155623e+03,132290.030752
2120,2584,2523,6.283611e+03,100904.279948


In [24]:
distance_threshold = list(result_1['distance'].quantile( q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]))

In [25]:
distance_threshold

[0.0,
 15132.638559499781,
 21402.830230160056,
 25920.399064785524,
 30364.88402831512,
 34719.66671076114,
 39915.95671982414,
 47372.45491507817,
 62890.4022520481,
 155899.8810224734]

In [26]:
result_1.loc[:,'level'] = 0

In [27]:
def get_level(dp, d):
    level_result = 1
    for ddpp in dp:
        if d > ddpp:
            level_result = level_result + 1
        else:
            break
    return level_result

In [28]:
for i in result_1.index:
    result_1.loc[i, 'level'] = get_level(  distance_threshold, result_1.loc[i, 'distance'])


In [29]:
result_1

,ozone_deso,dzone_deso,v_ij,distance,level
0,0114,0114,1.328431e+08,0.000000,1
1,0114,0115,2.557794e+07,18159.440980,3
2,0114,0117,4.094059e+06,36988.166911,7
3,0114,0120,1.368843e+07,56218.609477,9
4,0114,0123,4.911734e+05,10968.190328,2
...,...,...,...,...,...
2117,2583,2518,5.234224e+03,77605.749310,10
2118,2583,2583,9.534196e+05,0.000000,1
2119,2584,2521,4.155623e+03,132290.030752,10
2120,2584,2523,6.283611e+03,100904.279948,10


In [55]:
level_percent = 10
result_1_level = result_1[result_1['level'] == level_percent]

In [56]:
result_1_level

,ozone_deso,dzone_deso,v_ij,distance,level
25,0115,0136,3.487936e+05,63098.728534,10
52,0120,0125,4.541886e+06,65878.034274,10
55,0120,0128,2.189873e+05,63965.915019,10
82,0125,0120,4.541886e+06,65878.034274,10
129,0128,0120,2.189873e+05,63965.915019,10
...,...,...,...,...,...
2111,2582,2523,3.222649e+03,147195.193506,10
2114,2582,2581,9.954342e+01,77825.729404,10
2117,2583,2518,5.234224e+03,77605.749310,10
2119,2584,2521,4.155623e+03,132290.030752,10


In [57]:
Median = result_1_level['distance'].quantile( q=[0.5])
print(Median)

0.5    82023.929114
Name: distance, dtype: float64
